In [1]:
from __future__ import print_function
import sys
from syslog import LOG_MAIL

import os
from urllib3 import Retry
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import random
import argparse
import numpy as np

from math import log2
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, FloatSlider, IntSlider
from IPython.display import Image
import torch
%matplotlib inline

# UNICON Sharpening

In [2]:
def sharpening(labels, T):
    labels_sp = labels**(1/T)
    return labels_sp / labels_sp.sum(dim=0, keepdim=True)

# DINO Sharpening

In [13]:
def sharpening_DINO(labels, T):
    labels_sp = labels / T
    m = nn.Softmax(dim=0)
    return m(labels_sp)

# Visualization Sharpening

In [40]:
@interact
def visDensity(
    UNICON_T = FloatSlider(min=0., max=5., value=0.5, step=0.01),
    DINO_T = FloatSlider(min=0., max=5., value=0.06, step=0.01),
    inputX_0 = FloatSlider(min=0., max=1., value=0.4, step=0.01),
    inputX_1 = FloatSlider(min=0., max=1., value=0.2, step=0.01),
    inputX_2 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
    inputX_3 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
    inputX_4 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
    inputX_5 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
    inputX_6 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
    inputX_7 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
    inputX_8 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
    inputX_9 = FloatSlider(min=0., max=1., value=0.05, step=0.01),
): 
    num_class = 10
    inputX = [inputX_0, inputX_1, inputX_2,inputX_3,inputX_4,inputX_5,inputX_6,inputX_7,inputX_8,inputX_9]
    inputX = torch.FloatTensor(inputX)
    inputX_sp_UNICON = sharpening(inputX, UNICON_T)
    inputX_sp_DINO = sharpening_DINO(inputX, DINO_T)
    plt.figure(figsize=(24,6))
    plt.tight_layout()
    plt.subplot(131)
    plt.ylim(0,1)
    bars = plt.bar(range(0, 10), inputX.numpy())
    plt.bar_label(bars, fmt='%.2f')
    plt.title("Origin")
    plt.xticks(range(0, 10))
    plt.grid(True)
    plt.subplot(132)
    plt.ylim(0,1)
    bars = plt.bar(range(0, 10), inputX_sp_UNICON.numpy())
    plt.bar_label(bars, fmt='%.2f')
    plt.grid(True)
    plt.title("UNICON : " + str(UNICON_T))
    plt.xticks(range(0, 10))
    plt.subplot(133)
    plt.ylim(0,1)
    bars = plt.bar(range(0, 10), inputX_sp_DINO.numpy())
    plt.bar_label(bars, fmt='%.2f')
    plt.title("DINO T: " + str(DINO_T))
    plt.xticks(range(0, 10))
    plt.grid(True)
    plt.show()
    return 

interactive(children=(FloatSlider(value=0.5, description='UNICON_T', max=5.0, step=0.01), FloatSlider(value=0.…

備忘錄：
1. 預設參數
2. 相同強度 Tps 0.1
3. asym的情況 0.55 0.45 
4. 